# 1. Import Libraries and Tools and Create Helper Functions

In [1]:
! pip install chembl_webresource_client xgboost shap rdkit joblib matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 3.1 MB/s eta 0:00:00


In [2]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Optional, Tuple, Dict
from tqdm.auto import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, MACCSkeys, DataStructs
from rdkit.Chem.Descriptors import MolLogP

In [3]:
#=============================================================
# Helper Functions
#=============================================================

# Canonicalize smiles safely
def canonical_smiles(smiles):
    try:
        m = Chem.MolFromSmiles(smiles)
        if m is None:
            return None
        return Chem.MolToSmiles(m, isomericSmiles=True)
    except Exception:
        return None

# Convert standard value + unit to nM
def to_nM(val, unit):
    if pd.isna(val) or pd.isna(unit):
        return np.nan
    try:
        v = float(val)
    except:
        return np.nan
    u = unit.strip().lower()
    if 'nm' in u:
        return v
    if 'µm' in u or 'um' in u or 'μm' in u:
        return v * 1e3
    if u == 'm' or u == 'mol/l' or 'mol' in u:
        # assume M
        return v * 1e9
    # fallback: try to parse prefixes
    if u == 'mm':
        return v * 1e6
    return np.nan

# Convert IC50_nM TO pIC50
def to_pIC50(ic50_nM):
    # pIC50 = -log10(IC50 [M]) = 9 - log10(IC50 [nM])
    if ic50_nM <= 0 or pd.isna(ic50_nM):
        return np.nan
    return 9.0 - np.log10(ic50_nM)

In [4]:
def find_hiv_targets(client, search_term="protease"):
    """
    Search ChEMBL for HIV related targets.

    Parameters
    ----------
    client : chembl_webresource_client.new_client
        A ChEMBL new_client instance.
    search_term : str, optional
        Broad search term (default: "protease").

    Returns
    -------
    candidates : pd.DataFrame
        Filtered DataFrame of likely HIV/protease targets
        with chembl_id, pref_name, organism, and component_text.
    """

    tc = client.target
    hits = tc.search(search_term)

    rows = []
    for h in hits:
        tid = h.get('target_chembl_id')
        pname = h.get('pref_name') or ''
        org = h.get('organism') or ''

        comp_text = ''
        for c in h.get('target_components') or []:
            comp_text += ' ' + (c.get('component_description') or '')
            for s in c.get('target_component_synonyms') or []:
                comp_text += ' ' + (s.get('component_synonym') or '')

        rows.append({
            'chembl_id': tid,
            'pref_name': pname,
            'organism': org,
            'component_text': comp_text
        })

    targets_df = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

    # filter for HIV/protease-related targets
    mask = (
        targets_df['component_text'].str.lower().str.contains('hiv|human immunodeficiency', na=False) |
        targets_df['pref_name'].str.lower().str.contains(f'hiv|{search_term}', na=False) |
        targets_df['organism'].str.lower().str.contains('hiv|human immunodeficiency', na=False)
    )

    candidates = targets_df[mask].reset_index(drop=True)

    print(f"Found {len(candidates)} candidate target(s) mentioning 'HIV' or '{search_term}e'.")
    if len(candidates) == 0:
        print(f"No obvious HIV hits found. Try search terms like 'HIV-1 {search_term}' or by known accession.")
    return candidates[['chembl_id', 'pref_name', 'organism', 'component_text']]

# 2. Curate Data for HIV-1 integrase inhibitors

In [5]:
# ==============================================================================
# Search targets and show likely HIV integrase candidates
# ==============================================================================
candidates = find_hiv_targets(new_client, search_term="integrase")
display(candidates)

Found 9 candidate target(s) mentioning 'HIV' or 'integrasee'.


,chembl_id,pref_name,organism,component_text
0,CHEMBL3463,Human immunodeficiency virus type 2 integrase,Human immunodeficiency virus 2,Integrase pol Integrase
1,CHEMBL2366505,Integrase,Human immunodeficiency virus 1,Gag-Pol polyprotein Gag-Pol polyprotein Pr160...
2,CHEMBL4296304,Integrase,Human immunodeficiency virus 1,Gag-Pol polyprotein int Gag-Pol polyprotein P...
3,CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,Gag-Pol polyprotein pol Gag-Pol polyprotein P...
4,CHEMBL5823,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein gag-pol Gag-Pol polyprote...
5,CHEMBL3638326,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein gag-pol Gag-Pol polyprote...
6,CHEMBL3638331,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein gag-pol Gag-Pol polyprote...
7,CHEMBL3638352,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein gag-pol Gag-Pol polyprote...
8,CHEMBL3638360,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein gag-pol Gag-Pol polyprote...


In [6]:
# ================================================
# Fetch IC50 activities for chosen target(s)
# ================================================
activity_client = new_client.activity

# There are only two target HIV-1 proteases
target_ids = ['CHEMBL3471', 'CHEMBL2366505', 'CHEMBL4296304']

all_acts = []
for tid in target_ids:
    print("Downloading activities for", tid)
    # fetch activities filtered by standard_type=IC50 (iterable)
    acts_iter = activity_client.filter(target_chembl_id=tid, standard_type='IC50')
    acts = list(acts_iter)  # convert to list
    print(" -> returned", len(acts), "rows")
    for a in acts:
        a['_queried_target'] = tid
    all_acts.extend(acts)

print("Total activity rows collected:", len(all_acts))
acts_df = pd.DataFrame(all_acts)
# quick glance
display(acts_df.head())


 -> returned 8000 rows
 -> returned 1 rows
 -> returned 0 rows
Total activity rows collected: 8001


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,_queried_target
0,None,None,32538,[],CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",B,None,None,BAO_0000190,...,Human immunodeficiency virus type 1 integrase,11676,None,None,IC50,uM,UO_0000065,None,9.0,CHEMBL3471
1,None,None,34946,[],CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",B,None,None,BAO_0000190,...,Human immunodeficiency virus type 1 integrase,11676,None,None,IC50,uM,UO_0000065,None,1.4,CHEMBL3471
2,None,None,34947,[],CHEMBL701720,"Tested for inhibition of HIV-1 integrase, unde...",B,None,None,BAO_0000190,...,Human immunodeficiency virus type 1 integrase,11676,None,None,IC50,uM,UO_0000065,None,1.0,CHEMBL3471
3,None,None,34948,[],CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",B,None,None,BAO_0000190,...,Human immunodeficiency virus type 1 integrase,11676,None,None,IC50,uM,UO_0000065,None,1.7,CHEMBL3471
4,None,None,34949,[],CHEMBL701720,"Tested for inhibition of HIV-1 integrase, unde...",B,None,None,BAO_0000190,...,Human immunodeficiency virus type 1 integrase,11676,None,None,IC50,uM,UO_0000065,None,1.0,CHEMBL3471


In [7]:
acts_df.columns

Index(['action_type', 'activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type',
       'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value', '_queried_target'],
      dtype='object')

In [8]:
cols = ['molecule_chembl_id', 'canonical_smiles', 'standard_type', 'standard_value',
        'standard_units', 'pchembl_value', 'assay_chembl_id', 'assay_description',
        'target_chembl_id', 'target_pref_name', 'target_organism', 'standard_flag',
        'potential_duplicate', 'data_validity_comment', 'data_validity_description']
ii_df = acts_df[cols]
ii_df.head()

,molecule_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,pchembl_value,assay_chembl_id,assay_description,target_chembl_id,target_pref_name,target_organism,standard_flag,potential_duplicate,data_validity_comment,data_validity_description
0,CHEMBL324842,O=C(/C=C/c1ccc(O)c(O)c1)O[C@H](Cc1ccc(O)c(O)c1...,IC50,9000.0,nM,5.05,CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,1,1,None,None
1,CHEMBL304722,O=C(CC(O)CCc1ccc(O)c(O)c1)O[C@H]1Cc2cc(O)c(O)c...,IC50,1400.0,nM,5.85,CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,1,0,None,None
2,CHEMBL304722,O=C(CC(O)CCc1ccc(O)c(O)c1)O[C@H]1Cc2cc(O)c(O)c...,IC50,1000.0,nM,6.00,CHEMBL701720,"Tested for inhibition of HIV-1 integrase, unde...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,1,0,None,None
3,CHEMBL67076,O=C(CCc1ccc(O)c(O)c1)c1ccc(O)c(O)c1O,IC50,1700.0,nM,5.77,CHEMBL701719,"Inhibition of HIV-1 integrase, under 1 uM for ...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,1,0,None,None
4,CHEMBL67076,O=C(CCc1ccc(O)c(O)c1)c1ccc(O)c(O)c1O,IC50,1000.0,nM,6.00,CHEMBL701720,"Tested for inhibition of HIV-1 integrase, unde...",CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,1,0,None,None


In [9]:
# Remove rows where standard_value is not a number
non_numeric_mask = pd.to_numeric(ii_df['standard_value'], errors='coerce').isna()
non_numeric_rows = ii_df[non_numeric_mask]

print(f"Non-numeric rows: {len(non_numeric_rows)}")
display(non_numeric_rows.head())

# Now drop them
ii_df = ii_df[~non_numeric_mask].reset_index(drop=True)

Non-numeric rows: 115


,molecule_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,pchembl_value,assay_chembl_id,assay_description,target_chembl_id,target_pref_name,target_organism,standard_flag,potential_duplicate,data_validity_comment,data_validity_description
782,CHEMBL169404,CC(=O)NS(=O)(=O)c1ccc(NC(=O)c2ccccc2SC(=O)CCCC...,IC50,None,nM,None,CHEMBL702427,50% Inhibitory activity against Integrase; No ...,CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,0,0,None,None
783,CHEMBL169259,O=C(CCCC[n+]1ccccc1)Sc1ccccc1C(=O)Nc1ccc(S(=O)...,IC50,None,nM,None,CHEMBL702427,50% Inhibitory activity against Integrase; No ...,CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,0,0,None,None
784,CHEMBL169529,CC(=O)NS(=O)(=O)c1ccc(NC(=O)c2ccccc2SC(=O)CCCC...,IC50,None,nM,None,CHEMBL702427,50% Inhibitory activity against Integrase; No ...,CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,0,0,None,None
961,CHEMBL96959,Oc1cccc2ccc(/C=C/c3ccc4cccc(O)c4n3)nc12,IC50,None,nM,None,CHEMBL702102,In vitro anti-HIV integrase activity against i...,CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,0,0,None,None
968,CHEMBL97679,Oc1ccc(/C=C/c2ccc3ccccc3n2)cc1O,IC50,None,nM,None,CHEMBL879162,In vitro anti-HIV integrase activity against i...,CHEMBL3471,Human immunodeficiency virus type 1 integrase,Human immunodeficiency virus 1,0,0,None,None


In [10]:
ii_df['standard_units'].value_counts()

,count
standard_units,
nM,7819
ug.mL-1,67


In [11]:
print(ii_df['data_validity_comment'].value_counts())
print(ii_df['data_validity_description'].value_counts())
print(ii_df['potential_duplicate'].value_counts())

data_validity_comment
Outside typical range            1126
Potential transcription error       6
Name: count, dtype: int64
data_validity_description
Values for this activity type are unusually large/small, so may not be accurate                            1126
Values appear to be an order of magnitude different from previously reported, so units may be incorrect       6
Name: count, dtype: int64
potential_duplicate
0    7501
1     385
Name: count, dtype: int64


In [12]:
# keep rows where potential_duplicate is not True
ii_df = ii_df[ii_df['potential_duplicate'] != True]

# keep rows where data_validity_comment is null/NaN
ii_df = ii_df[ii_df['data_validity_comment'].isna()]

ii_df.shape

(6392, 15)

In [13]:
print(ii_df['data_validity_comment'].value_counts())
print(ii_df['data_validity_description'].value_counts())
print(ii_df['potential_duplicate'].value_counts())

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
potential_duplicate
0    6392
Name: count, dtype: int64


In [14]:
ii_df.to_parquet('ii_df.parquet', index=False)

# 3. Standardize IC50 and Deduplicate the data

In [15]:
# normalize unit strings
def norm_unit(u):
    if pd.isna(u):
        return u
    if u == 'ug.mL-1':
        return 'ug/mL'
    return u


# compute mol weight for each unique canonical_smiles
def molwt_from_smiles(smi):
    if pd.isna(smi):
        return np.nan
    try:
        m = Chem.MolFromSmiles(smi)
        if m is None:
            return np.nan
        return Descriptors.MolWt(m)  # g/mol
    except:
        return np.nan


# function to convert a row to nM
def value_to_nM(row):
    val = row['standard_value']
    unit = row['standard_units_norm']
    # ensure numeric
    try:
        v = float(val)
    except:
        return np.nan
    if pd.isna(unit):
        return np.nan

    if unit == 'nM':
        return v

    elif unit == 'ug/mL':
        mw = row.get('mol_wt_g_per_mol', np.nan)
        if pd.isna(mw) or mw == 0:
            return np.nan
        # nM = v * 1e6 / MW
        return float(v) * 1e6 / float(mw)

    # add more conversions if you see additional units
    return np.nan


# compute pIC50
def to_pIC50(nM):
    if pd.isna(nM) or nM <= 0:
        return np.nan
    return 9.0 - np.log10(nM)

In [16]:
ii_df = pd.read_parquet('ii_df.parquet')

In [17]:
print(ii_df.shape)

(6392, 15)


In [18]:
ii_df['standard_units_norm'] = ii_df['standard_units'].astype(str).apply(norm_unit)

# compute MW per unique smiles and map back
unique_smiles = ii_df['canonical_smiles'].dropna().unique().tolist()
mw_map = {}
for s in unique_smiles:
    mw_map[s] = molwt_from_smiles(s)

# add MW column
ii_df['mol_wt_g_per_mol'] = ii_df['canonical_smiles'].map(mw_map)

ii_df['IC50_nM'] = ii_df.apply(value_to_nM, axis=1)

ii_df['pIC50'] = ii_df['IC50_nM'].apply(to_pIC50)

# Quick diagnostics: show counts by unit and how many converted sucessfully
print("Unit value counts:")
print(ii_df['standard_units_norm'].value_counts(dropna=False))
print("\nConversion success counts (IC50_nM not null):")
print(ii_df.groupby('standard_units_norm')['IC50_nM'].apply(lambda s: s.notna().sum()))

# Flag rows where conversion failed (mass units but no MW)
mask_mass_units = ii_df['standard_units_norm'].str.contains('ug|mg', na=False)
failed = ii_df[mask_mass_units & ii_df['IC50_nM'].isna()]
print("\nRows with mass units that could not be converted (need manual inspection):", len(failed))
display(failed.head(10))

Unit value counts:
standard_units_norm
nM       6353
ug/mL      39
Name: count, dtype: int64

Conversion success counts (IC50_nM not null):
standard_units_norm
nM       6353
ug/mL      39
Name: IC50_nM, dtype: int64

Rows with mass units that could not be converted (need manual inspection): 0


,molecule_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,pchembl_value,assay_chembl_id,assay_description,target_chembl_id,target_pref_name,target_organism,standard_flag,potential_duplicate,data_validity_comment,data_validity_description,standard_units_norm,mol_wt_g_per_mol,IC50_nM,pIC50


In [19]:
ii_df['standard_units_norm'].value_counts()

,count
standard_units_norm,
nM,6353
ug/mL,39


In [20]:
# Drop rows where standard_units_norm is None (or missing)
ii_df_clean = ii_df[ii_df['standard_units_norm']!='None']
ii_df_clean['standard_units_norm'].value_counts()

,count
standard_units_norm,
nM,6353
ug/mL,39


In [21]:
ii_df_clean.drop(columns=['standard_units', 'standard_flag', 'potential_duplicate',
                          'data_validity_comment', 'data_validity_description', 'mol_wt_g_per_mol'],
                 inplace=True)

print(ii_df_clean.shape)

(6392, 13)


In [22]:
ii_df_clean.to_parquet('ii_df_clean.parquet', index=False)

# 4. Compute Descriptors and Fingerprints

In [23]:
# ------------------------------------------------------------------------------
# Utilities: SMILES -> Mol
# ------------------------------------------------------------------------------
def mol_from_smiles_safe(smi: str) -> Optional[Chem.Mol]:
    """Return RDKit Mol for a SMILES string or None if parsing fails."""
    if smi is None or (isinstance(smi, float) and np.isnan(smi)):
        return None
    try:
        m = Chem.MolFromSmiles(str(smi))
        return m
    except Exception:
        return None

def add_rdkit_mols(df: pd.DataFrame, smiles_col: str = 'canonical_smiles', mol_col: str = 'rdkit_mol',
                   show_progress: bool = False) -> pd.DataFrame:
    """Add a column with RDKit Mol objects parsed from SMILES. Returns new DataFrame (copy)."""
    df = df.copy()
    seq = df[smiles_col].tolist()
    mols = []
    if show_progress:
        iterable = tqdm(seq, desc='Parsing SMILES')
    else:
        iterable = seq
    for s in iterable:
        mols.append(mol_from_smiles_safe(s))
    df[mol_col] = mols
    return df

# ------------------------------------------------------------------------------
# Descriptor calculation
# ------------------------------------------------------------------------------
DEFAULT_DESC_LIST = [
    'MolWt', 'MolLogP', 'MolMR', 'TPSA', 'NumHDonors', 'NumHAcceptors',
    'NumRotatableBonds', 'NumAromaticRings', 'HeavyAtomCount', 'FractionCSP3'
]

def compute_descriptors_for_mol(m: Chem.Mol) -> Dict[str, float]:
    """Compute a set of basic 2D descriptors for a single RDKit Mol."""
    if m is None:
        return {k: np.nan for k in DEFAULT_DESC_LIST}
    try:
        molwt = Descriptors.MolWt(m)
        logp, mr = rdMolDescriptors.CalcCrippenDescriptors(m)
        tpsa = rdMolDescriptors.CalcTPSA(m)
        hbd = rdMolDescriptors.CalcNumHBD(m)
        hba = rdMolDescriptors.CalcNumHBA(m)
        rot = rdMolDescriptors.CalcNumRotatableBonds(m)
        arom = rdMolDescriptors.CalcNumAromaticRings(m)
        hac = Descriptors.HeavyAtomCount(m)
        fsp3 = rdMolDescriptors.CalcFractionCSP3(m)
        return {
            'MolWt': float(molwt),
            'MolLogP': float(logp),
            'MolMR': float(mr),
            'TPSA': float(tpsa),
            'NumHDonors': int(hbd),
            'NumHAcceptors': int(hba),
            'NumRotatableBonds': int(rot),
            'NumAromaticRings': int(arom),
            'HeavyAtomCount': int(hac),
            'FractionCSP3': float(fsp3)
        }
    except Exception:
        return {k: np.nan for k in DEFAULT_DESC_LIST}

def compute_descriptors_df(df: pd.DataFrame, mol_col: str = 'rdkit_mol',
                           desc_names: Optional[List[str]] = None,
                           show_progress: bool = False) -> pd.DataFrame:
    """Compute descriptors for all molecules in df[mol_col] and return a DataFrame aligned with df index."""
    if desc_names is None:
        desc_names = DEFAULT_DESC_LIST
    rows = []
    seq = df[mol_col].tolist()
    if show_progress:
        iterable = tqdm(seq, desc='Computing descriptors')
    else:
        iterable = seq
    for m in iterable:
        rows.append(compute_descriptors_for_mol(m))
    desc_df = pd.DataFrame(rows, index=df.index)[desc_names]
    return desc_df

# ------------------------------------------------------------------------------
# Fingerprint calculations
# ------------------------------------------------------------------------------
def morgan_fp_array(m: Chem.Mol, radius: int = 2, nBits: int = 2048) -> np.ndarray:
    """Return numpy array (0/1 int) of Morgan fingerprint for a single RDKit Mol."""
    if m is None:
        return np.zeros((nBits,), dtype=np.uint8)
    try:
        bitvect = AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=nBits)
        arr = np.zeros((nBits,), dtype=np.uint8)
        DataStructs.ConvertToNumpyArray(bitvect, arr)
        return arr
    except Exception:
        return np.zeros((nBits,), dtype=np.uint8)

def compute_fingerprints_arrays(df: pd.DataFrame, mol_col: str = 'rdkit_mol',
                                morgan_radius: int = 2, morgan_nbits: int = 512,
                                show_progress: bool = False
                               ) -> Tuple[np.ndarray, Optional[np.ndarray]]:
    """Compute Morgan fingerprint array (n_samples, nBits). Optionally compute MACCS (n_samples, 167).
       Returns morgan_array."""
    mols = df[mol_col].tolist()
    if show_progress:
        mols_iter = tqdm(mols, desc='Computing fingerprints')
    else:
        mols_iter = mols
    morgan_list = [morgan_fp_array(m, radius=morgan_radius, nBits=morgan_nbits) for m in mols_iter]
    morgan_arr = np.vstack(morgan_list).astype(np.uint8)
    return morgan_arr

# ------------------------------------------------------------------------------
# Helpers to convert fp arrays -> DataFrame with bit columns
# ------------------------------------------------------------------------------
def fp_array_to_df(fp_arr: np.ndarray, prefix: str = 'FP') -> pd.DataFrame:
    """Convert 2D numpy fingerprint array (n_samples, nBits) to DataFrame with columns FP_0..FP_{nBits-1}."""
    nBits = fp_arr.shape[1]
    cols = [f'{prefix}_{i}' for i in range(nBits)]
    return pd.DataFrame(fp_arr, columns=cols, index=range(fp_arr.shape[0]))

# ------------------------------------------------------------------------------
# Build feature DataFrame
# ------------------------------------------------------------------------------
def build_feature_dataframe(df: pd.DataFrame,
                            smiles_col: str = 'canonical_smiles',
                            mol_col: str = 'rdkit_mol',
                            desc_names: Optional[List[str]] = None,
                            morgan_radius: int = 2,
                            morgan_nbits: int = 512,
                            show_progress: bool = False
                           ) -> Tuple[pd.DataFrame, np.ndarray, Optional[np.ndarray]]:
    """Given a DataFrame with SMILES, return:
       - feature_df: descriptors + fingerprint bit columns (pandas DataFrame)
       - morgan_array: (n_samples, nBits) numpy array
    """
    # 1) ensure rdkit mols exist
    if mol_col not in df.columns:
        df = add_rdkit_mols(df, smiles_col=smiles_col, mol_col=mol_col, show_progress=show_progress)
    # 2) descriptors
    desc_df = compute_descriptors_df(df, mol_col=mol_col, desc_names=desc_names, show_progress=show_progress)
    # 3) fingerprints arrays
    morgan_arr= compute_fingerprints_arrays(df, mol_col=mol_col,
                                            morgan_radius=morgan_radius, morgan_nbits=morgan_nbits,
                                            show_progress=show_progress)
    # 4) convert fp arrays to DataFrame
    morgan_df = fp_array_to_df(morgan_arr, prefix=f'Morgan_{morgan_nbits}')
    feature_df = pd.concat([desc_df.reset_index(drop=True), morgan_df.reset_index(drop=True)], axis=1)
    return feature_df, morgan_arr

# ------------------------------------------------------------------------------
# Save/load helpers
# ------------------------------------------------------------------------------
def save_feature_arrays(prefix: str,
                        feature_df: pd.DataFrame,
                        morgan_arr: np.ndarray,
                        ) -> Dict[str,str]:
    """Save feature DataFrame to parquet and fingerprint arrays as .npy files.
       Returns dict of saved paths.
    """
    feature_path = f'{prefix}_features.parquet'
    morgan_path = f'{prefix}_morgan.npy'
    pd.DataFrame(feature_df).to_parquet(feature_path, index=False)
    np.save(morgan_path, morgan_arr)
    saved = {'features_parquet': feature_path, 'morgan_npy': morgan_path}
    return saved

def load_feature_arrays(prefix: str) -> Tuple[pd.DataFrame, np.ndarray, Optional[np.ndarray]]:
    """Load previously saved files (expects same naming used in save_feature_arrays)."""
    feature_path = f'{prefix}_features.parquet'
    morgan_path = f'{prefix}_morgan.npy'
    feature_df = pd.read_parquet(feature_path)
    morgan_arr = np.load(morgan_path)
    return feature_df, morgan_arr

# ------------------------------------------------------------------------------
# Example quick diagnostics
# ------------------------------------------------------------------------------
def fingerprint_density_stats(morgan_arr: np.ndarray) -> Dict[str, float]:
    """Return simple stats: mean bit density per fingerprint, fraction of bits with >0 frequency."""
    if morgan_arr is None:
        return {}
    # average number of bits set per sample
    bits_per_sample = morgan_arr.sum(axis=1)
    mean_bits = float(bits_per_sample.mean())
    # fraction of fingerprint bits that are ever on
    bits_active = (morgan_arr.sum(axis=0) > 0).mean()
    return {'mean_bits_set_per_sample': mean_bits, 'fraction_bits_active': float(bits_active)}

# ------------------------------------------------------------------------------
# Convenience: aggregate duplicates by SMILES (median target)
# ------------------------------------------------------------------------------
def aggregate_by_smiles(df: pd.DataFrame, smiles_col: str = 'canonical_smiles',
                        value_col: str = 'pIC50') -> pd.DataFrame:
    """Aggregate rows by canonical SMILES, returning DataFrame with unique smiles and median value_col.
       Keeps the first molecule_chembl_id if present.
    """
    df = df.copy()
    df['can_smiles'] = df[smiles_col]
    agg = df.groupby('can_smiles').agg({
        value_col: 'median',
        'molecule_chembl_id': lambda x: x.dropna().astype(str).unique().tolist()
    }).rename(columns={value_col: f'median_{value_col}'}).reset_index()
    return agg

In [24]:
ii_df = pd.read_parquet('ii_df_clean.parquet')

In [25]:
integrase = ii_df.copy()

# Add RDKit mols (safe)
integrase = add_rdkit_mols(integrase, smiles_col='canonical_smiles', mol_col='rdkit_mol', show_progress=True)

# Drop rows where SMILES failed
integrase = integrase[integrase['rdkit_mol'].notna()].reset_index(drop=True)

# Build features
feature_inte, morgan_arr = build_feature_dataframe(integrase, smiles_col='canonical_smiles',
                                                morgan_radius=2, morgan_nbits=512,
                                                show_progress=True)

# attach pIC50 to features for modelling
feature_inte['pIC50'] = integrase['pIC50'].values

# Save
paths = save_feature_arrays('integrase_qsar', feature_inte, morgan_arr)
print(paths)

Parsing SMILES:   0%|          | 0/6392 [00:00<?, ?it/s]

Computing descriptors:   0%|          | 0/6386 [00:00<?, ?it/s]

Computing fingerprints:   0%|          | 0/6386 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:51:52] DEPRECATION WARNING: please use MorganGenerator
[15:5

{'features_parquet': 'integrase_qsar_features.parquet', 'morgan_npy': 'integrase_qsar_morgan.npy'}


[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerator
[15:51:53] DEPRECATION WARNING: please use MorganGenerat

# 5. QSAR Model for integrase inhibitor pIC50 value

We have appended the morgan fingerprints and molecular descriptors into a single dataframe

In [26]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.3 MB/s eta 0:00:00


In [27]:
# Requirements: pandas, numpy, scikit-learn, xgboost, joblib, matplotlib
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

# Models for scaled data
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Tree based models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [28]:
df = pd.read_parquet('integrase_qsar_features.parquet')
print("rows, cols:", df.shape)

rows, cols: (6386, 523)


In [29]:
df.head()

,MolWt,MolLogP,MolMR,TPSA,NumHDonors,NumHAcceptors,NumRotatableBonds,NumAromaticRings,HeavyAtomCount,FractionCSP3,...,Morgan_512_503,Morgan_512_504,Morgan_512_505,Morgan_512_506,Morgan_512_507,Morgan_512_508,Morgan_512_509,Morgan_512_510,Morgan_512_511,pIC50
0,498.484,2.8920,125.2126,177.14,7,10,7,3,36,0.269231,...,0,1,0,0,1,0,0,0,0,5.853872
1,498.484,2.8920,125.2126,177.14,7,10,7,3,36,0.269231,...,0,1,0,0,1,0,0,0,0,6.000000
2,290.271,2.0301,73.8595,118.22,5,6,4,2,21,0.133333,...,0,0,0,0,0,0,0,0,0,5.769551
3,290.271,2.0301,73.8595,118.22,5,6,4,2,21,0.133333,...,0,0,0,0,0,0,0,0,0,6.000000
4,516.455,1.0296,125.1976,211.28,7,11,7,2,37,0.240000,...,0,0,1,0,1,0,0,0,0,6.602060


In [30]:
# Prepare X and y
y = df['pIC50']
X = df.drop(columns=['pIC50'])
print('Feature count: ', X.shape[1])

Feature count:  522


In [31]:
y.isna().sum()

np.int64(0)

In [32]:
# ------------------------------------------------------------------------------
# Quick feature cleanup
# ------------------------------------------------------------------------------

# a. remove near-constant features

# Fit the selector
vt = VarianceThreshold(threshold=1e-6)
X_v = vt.fit_transform(X)
# Features kept
kept_features = X.columns[vt.get_support()].tolist()
# Features removed
removed_features = [col for col in X.columns if col not in kept_features]
# Create the reduced DataFrame
X = pd.DataFrame(X_v, columns=kept_features)

print(f"Features after variance filter: {len(kept_features)}")
print(f"Features removed: {len(removed_features)}")
# Optional: display or save the removed features
print("\nRemoved features (first 20):")
print(removed_features[:20])

Features after variance filter: 522
Features removed: 0

Removed features (first 20):
[]


In [33]:
# b. (optional) remove extremely collinear features (simple correlation filter)

corr_thresh = 0.98
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > corr_thresh)]
if to_drop:
    X.drop(columns=to_drop, inplace=True)
    print("Dropped highly correlated features:", to_drop)

Dropped highly correlated features: ['MolMR', 'HeavyAtomCount']


In [34]:
X.shape

(6386, 520)

In [35]:
# ------------------------------------------------------------------------------
# Define descriptor columns to scale and fingerprint columns to pass through
# ------------------------------------------------------------------------------
descriptors = ['MolWt', 'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors',
    'NumRotatableBonds', 'NumAromaticRings', 'FractionCSP3']
fingerprints = [c for c in X.columns if c not in descriptors]

# ColumnTransformer: scale descriptors, pass fingerprints through
preprocessor = ColumnTransformer([
    ("desc", StandardScaler(), descriptors),
    ("fp", "passthrough", fingerprints)
])

In [36]:
# ---------------------------
# k-fold OOF trainer
# ---------------------------
def kfold_train_predict(model, X_df, y_arr, n_splits=5, random_state=42):
    """
    model: estimator or pipeline (should accept fit/predict)
    X_df: pandas DataFrame (full dataset)
    y_arr: numpy array or Series
    returns: oof_preds (np.array same length as X_df), mean_r2, mean_rmse
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof_preds = np.zeros(len(X_df))
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_df, y_arr), 1):
        X_train, X_val = X_df.iloc[train_idx], X_df.iloc[val_idx]
        y_train, y_val = y_arr[train_idx], y_arr[val_idx]

        # fit on train fold, predict on val fold
        model.fit(X_train, y_train)
        preds = model.predict(X_val)

        oof_preds[val_idx] = preds

        fold_r2 = r2_score(y_val, preds)
        fold_rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_scores.append((fold_r2, fold_rmse))
        print(f"Fold {fold}: R2 = {fold_r2:.3f}, RMSE = {fold_rmse:.3f}")

    mean_r2 = np.mean([s[0] for s in fold_scores])
    mean_rmse = np.mean([s[1] for s in fold_scores])
    print(f"\nMean CV R2 = {mean_r2:.3f}, Mean RMSE = {mean_rmse:.3f}")
    return oof_preds, mean_r2, mean_rmse

In [37]:
# ---------------------------
# Define models and pipelines
# ---------------------------
models = {}

# Linear models (need scaling) -> include preprocessor
models['Linear'] = Pipeline([("preproc", preprocessor), ("model", LinearRegression())])
models['ElasticNet'] = Pipeline([("preproc", preprocessor), ("model", ElasticNet(random_state=42, max_iter=5000))])

# SVR (needs scaling)
models['SVM'] = Pipeline([("preproc", preprocessor), ("model", SVR())])

# KNN
models['KNN'] = Pipeline([("preproc", preprocessor), ("model", KNeighborsRegressor())])

# MLP
models['MLP'] = Pipeline([("preproc", preprocessor), ("model", MLPRegressor(max_iter=2000, random_state=42))])

# Random Forest (trees don't need scaling, but pipeline is fine)
models['Random Forest'] = Pipeline([("preproc", preprocessor), ("model", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))])

# XGBoost
models['XGBoost'] = Pipeline([("preproc", preprocessor), ("model", XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1, objective='reg:squarederror'))])

In [38]:
# ---------------------------
# Run k-fold for each model
# ---------------------------
results = {}
oof_dict = {}

for name, model in models.items():
    print(f"\n=== Training model: {name} ===")
    oof_preds, mean_r2, mean_rmse = kfold_train_predict(model, X, y, n_splits=5, random_state=42)
    results[name] = {"r2": mean_r2, "rmse": mean_rmse}
    oof_dict[name] = oof_preds

# Summary
print("\nSummary (mean CV results):")
for name, res in results.items():
    print(f"{name}: R2 = {res['r2']:.4f}, RMSE = {res['rmse']:.4f}")


=== Training model: Linear ===
Fold 1: R2 = 0.632, RMSE = 0.811
Fold 2: R2 = 0.650, RMSE = 0.818
Fold 3: R2 = 0.631, RMSE = 0.838
Fold 4: R2 = 0.618, RMSE = 0.836
Fold 5: R2 = 0.653, RMSE = 0.797

Mean CV R2 = 0.637, Mean RMSE = 0.820

=== Training model: ElasticNet ===
Fold 1: R2 = -0.001, RMSE = 1.337
Fold 2: R2 = -0.001, RMSE = 1.382
Fold 3: R2 = -0.001, RMSE = 1.380
Fold 4: R2 = -0.000, RMSE = 1.352
Fold 5: R2 = -0.000, RMSE = 1.352

Mean CV R2 = -0.001, Mean RMSE = 1.361

=== Training model: SVM ===
Fold 1: R2 = 0.732, RMSE = 0.692
Fold 2: R2 = 0.763, RMSE = 0.672
Fold 3: R2 = 0.745, RMSE = 0.697
Fold 4: R2 = 0.739, RMSE = 0.691
Fold 5: R2 = 0.761, RMSE = 0.661

Mean CV R2 = 0.748, Mean RMSE = 0.683

=== Training model: KNN ===
Fold 1: R2 = 0.708, RMSE = 0.722
Fold 2: R2 = 0.752, RMSE = 0.688
Fold 3: R2 = 0.742, RMSE = 0.700
Fold 4: R2 = 0.714, RMSE = 0.723
Fold 5: R2 = 0.762, RMSE = 0.660

Mean CV R2 = 0.736, Mean RMSE = 0.699

=== Training model: MLP ===
Fold 1: R2 = 0.643, RMS

# 6. Tune the Best Models

### Support Vector Machine (SVR),
### K-Nearest Neighbours (KNN),
### Random Forest (RF)

In [39]:
import optuna
import joblib
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, loguniform
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

In [40]:
# PCA variant of the model
n_fp = len(fingerprints)
print(f"Using {len(descriptors)} descriptor columns and {n_fp} fingerprint columns.")

if n_fp == 0:
    raise ValueError("No fingerprint columns detected. Make sure your X contains fingerprint bit columns.")

# ---------- CV settings ----------
cv_outer = KFold(n_splits=5, shuffle=True, random_state=42)

Using 8 descriptor columns and 512 fingerprint columns.


In [41]:
# ----------------------- SVM ---------------------------------
def run_optuna_svm(n_trials=80):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        # SVR hyperparameters (log-uniform sampling where appropriate)
        kernel = trial.suggest_categorical("kernel", ["rbf", "poly", "sigmoid"])
        C = trial.suggest_loguniform("C", 1e-3, 1e3)
        epsilon = trial.suggest_loguniform("epsilon", 1e-4, 1.0)
        gamma_choice = trial.suggest_categorical("gamma_choice", ["scale", "auto", "numeric"])
        if gamma_choice == "numeric":
            gamma = trial.suggest_loguniform("gamma", 1e-5, 1e1)
        else:
            gamma = gamma_choice
        # degree and coef0 for polynomial kernel
        if kernel == "poly":
            degree = trial.suggest_int("degree", 2, 5)
            coef0 = trial.suggest_float("coef0", 0.0, 1.0)
        else:
            degree = 3
            coef0 = 0.0

        svr = SVR(kernel=kernel, C=C, epsilon=epsilon, gamma=gamma, degree=degree, coef0=coef0, max_iter=100000)
        pipe = Pipeline([("preproc", preprocessor), ("svr", svr)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    # reconstruct SVR
    best_kernel = best.get("kernel", "rbf")
    best_gamma = best.get("gamma", best.get("gamma_choice", "scale"))
    if best_gamma == "numeric":
        best_gamma = best.get("gamma")
    best_degree = best.get("degree", 3)
    best_coef0 = best.get("coef0", 0.0)

    best_svm = SVR(
        kernel=best_kernel,
        C=best.get("C"),
        epsilon=best.get("epsilon"),
        gamma=best_gamma,
        degree=best_degree,
        coef0=best_coef0,
        max_iter=100000
    )
    best_pipe = Pipeline([("preproc", preprocessor_best), ("svr", best_svm)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "svm_optuna_pipeline.joblib")
    print("SVM best R2:", study.best_value)
    print("SVM best params:", study.best_params)
    return study, best_pipe

In [42]:
# ------------------ KNN -----------------------
def run_optuna_knn(n_trials=50):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
        weights = trial.suggest_categorical("weights", ["uniform", "distance"])
        p = trial.suggest_int("p", 1, 2)  # 1 = manhattan, 2 = euclidean

        knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p)
        pipe = Pipeline([("preproc", preprocessor), ("knn", knn)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    best_knn = KNeighborsRegressor(n_neighbors=best["n_neighbors"], weights=best["weights"], p=best["p"])
    best_pipe = Pipeline([("preproc", preprocessor_best), ("knn", best_knn)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "knn_optuna_pipeline.joblib")
    print("KNN best R2:", study.best_value)
    print("KNN best params:", study.best_params)
    return study, best_pipe

In [43]:
# ------------------------ MLP ------------------------------
def run_optuna_mlp(n_trials=50):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 500)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        # MLP hyperparams
        n_layers = trial.suggest_int("n_layers", 1, 3)
        hidden_sizes = []
        for i in range(n_layers):
            hidden_sizes.append(trial.suggest_int(f"n_units_l{i}", 32, 512))
        hidden_tuple = tuple(hidden_sizes)
        activation = trial.suggest_categorical("activation", ["relu", "tanh"])
        alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-1)
        learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-5, 1e-1)
        max_iter = 2000

        mlp = MLPRegressor(hidden_layer_sizes=hidden_tuple, activation=activation,
                           alpha=alpha, learning_rate_init=learning_rate_init,
                           max_iter=max_iter, random_state=42)
        pipe = Pipeline([("preproc", preprocessor), ("mlp", mlp)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])

    n_layers = best["n_layers"]
    hidden_sizes = tuple(best[f"n_units_l{i}"] for i in range(n_layers))
    best_mlp = MLPRegressor(hidden_layer_sizes=hidden_sizes,
                            activation=best["activation"],
                            alpha=best["alpha"],
                            learning_rate_init=best["learning_rate_init"],
                            max_iter=2000,
                            random_state=42)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("mlp", best_mlp)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "mlp_optuna_pipeline.joblib")
    print("MLP best R2:", study.best_value)
    print("MLP best params:", study.best_params)
    return study, best_pipe

In [49]:
# -------------------- Random Forest --------------------------------
def run_optuna_rf(n_trials=50):
    def objective(trial):
        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", "passthrough", fingerprints)
        ])

        n_estimators = trial.suggest_int("n_estimators", 100, 500)
        max_depth = trial.suggest_int("max_depth", 3, 50)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3, 0.5, 0.8])

        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   max_features=max_features, n_jobs=-1, random_state=42)
        pipe = Pipeline([("preproc", preprocessor), ("rf", rf)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", "passthrough", fingerprints)
    ])
    best_rf = RandomForestRegressor(n_estimators=best["n_estimators"], max_depth=best["max_depth"],
                                    min_samples_split=best["min_samples_split"],
                                    min_samples_leaf=best["min_samples_leaf"],
                                    max_features=best["max_features"], n_jobs=-1, random_state=42)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("rf", best_rf)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "rf_optuna_pipeline.joblib")
    print("RF best R2:", study.best_value)
    print("RF best params:", study.best_params)
    return study, best_pipe

In [45]:
# ------------------------ XGBoost ---------------------------------
def run_optuna_xgb(n_trials=80, use_gpu=False):
    # Set tree_method to gpu_hist if use_gpu True (ensure XGBoost GPU support)
    def objective(trial):
        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", "passthrough", fingerprints)
        ])

        param = {
            "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
            "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
            "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
            "subsample": trial.suggest_float("subsample", 0.4, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 0.3),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
            "gamma": trial.suggest_loguniform("gamma", 1e-8, 10.0)
        }
        if use_gpu:
            param["tree_method"] = "gpu_hist"

        model = XGBRegressor(objective="reg:squarederror", random_state=42, n_jobs=-1, **param)
        pipe = Pipeline([("preproc", preprocessor), ("xgb", model)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", "passthrough", fingerprints)
    ])

    # reconstruct best xgb with best params
    if use_gpu:
        best["tree_method"] = "gpu_hist"
    best_xgb = XGBRegressor(objective="reg:squarederror", random_state=42, n_jobs=-1, **best)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("xgb", best_xgb)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "xgb_optuna_pipeline.joblib")
    print("XGB best R2:", study.best_value)
    print("XGB best params:", study.best_params)
    return study, best_pipe

In [46]:
# ----------------------------
# Example: run all studies sequentially (adjust n_trials)
# ----------------------------
svm_study, svm_pipe = run_optuna_svm(n_trials=50)

[I 2025-10-05 15:57:48,721] A new study created in memory with name: no-name-013136ec-3d11-4a71-af96-04bb79f27196


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-05 15:58:31,641] Trial 0 finished with value: 0.5939899885254347 and parameters: {'use_pca': False, 'kernel': 'rbf', 'C': 0.07988333033018587, 'epsilon': 0.3180549068868418, 'gamma_choice': 'scale'}. Best is trial 0 with value: 0.5939899885254347.
[I 2025-10-05 15:58:40,365] Trial 1 finished with value: 0.5910852429562989 and parameters: {'use_pca': True, 'pca_n_components': 14, 'kernel': 'rbf', 'C': 652.8953107301616, 'epsilon': 0.08248104844487571, 'gamma_choice': 'numeric', 'gamma': 1.9685456885326376}. Best is trial 0 with value: 0.5939899885254347.
[I 2025-10-05 15:59:30,120] Trial 2 finished with value: -3.3029127552697033 and parameters: {'use_pca': False, 'kernel': 'sigmoid', 'C': 12.691198216141695, 'epsilon': 0.05799213688108426, 'gamma_choice': 'auto'}. Best is trial 0 with value: 0.5939899885254347.
[I 2025-10-05 16:00:19,225] Trial 3 finished with value: 0.46768062140277245 and parameters: {'use_pca': False, 'kernel': 'poly', 'C': 0.30646151818912737, 'epsilon':

In [47]:
knn_study, knn_pipe = run_optuna_knn(n_trials=50)

[I 2025-10-05 16:19:13,271] A new study created in memory with name: no-name-74033c5f-e098-4995-b301-e28b403e0ddd


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-05 16:19:14,446] Trial 0 finished with value: 0.6827252416897578 and parameters: {'use_pca': False, 'n_neighbors': 2, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 0.6827252416897578.
[I 2025-10-05 16:19:16,584] Trial 1 finished with value: 0.7086095585554057 and parameters: {'use_pca': True, 'pca_n_components': 35, 'n_neighbors': 17, 'weights': 'uniform', 'p': 1}. Best is trial 1 with value: 0.7086095585554057.
[I 2025-10-05 16:19:27,436] Trial 2 finished with value: 0.7148907566765257 and parameters: {'use_pca': False, 'n_neighbors': 14, 'weights': 'uniform', 'p': 1}. Best is trial 2 with value: 0.7148907566765257.
[I 2025-10-05 16:19:33,890] Trial 3 finished with value: 0.7113389895046133 and parameters: {'use_pca': True, 'pca_n_components': 178, 'n_neighbors': 23, 'weights': 'distance', 'p': 1}. Best is trial 2 with value: 0.7148907566765257.
[I 2025-10-05 16:19:45,135] Trial 4 finished with value: 0.703440390648223 and parameters: {'use_pca': False, 'n_nei

In [50]:
rf_study, rf_pipe = run_optuna_rf(n_trials=50)

[I 2025-10-05 16:54:26,726] A new study created in memory with name: no-name-3f43627a-e46d-4717-9e04-c67b73966a4d


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-05 16:54:40,595] Trial 0 finished with value: 0.7444270216090687 and parameters: {'n_estimators': 208, 'max_depth': 46, 'min_samples_split': 11, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7444270216090687.
[I 2025-10-05 16:56:00,879] Trial 1 finished with value: 0.749373075588142 and parameters: {'n_estimators': 400, 'max_depth': 26, 'min_samples_split': 12, 'min_samples_leaf': 4, 'max_features': 0.3}. Best is trial 1 with value: 0.749373075588142.
[I 2025-10-05 16:56:14,284] Trial 2 finished with value: 0.7491634735318237 and parameters: {'n_estimators': 387, 'max_depth': 42, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 1 with value: 0.749373075588142.
[I 2025-10-05 16:56:22,158] Trial 3 finished with value: 0.5532320591059305 and parameters: {'n_estimators': 471, 'max_depth': 27, 'min_samples_split': 17, 'min_samples_leaf': 18, 'max_features': 'log2'}. Best is trial 1 with value: 0.749373075588142.

# 7. Try a meta learner or ensemble of the best models

In [51]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict

# reproducible folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 1) Get OOF predictions for each base model (these preds are made by models
#    that were trained without the corresponding sample — no leakage)
print("Generating OOF preds (this may take time)...")
oof_knn = cross_val_predict(knn_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_svm = cross_val_predict(svm_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_rf  = cross_val_predict(rf_pipe,  X, y, cv=kf, n_jobs=-1, method='predict')

# Stack OOF predictions (n_samples x n_models)
stack_oof = np.vstack([oof_knn, oof_svm, oof_rf]).T

# 2) Simple average ensemble
ens_mean = stack_oof.mean(axis=1)
r2_mean = r2_score(y, ens_mean)
rmse_mean = np.sqrt(mean_squared_error(y, ens_mean))
print(f"Simple average ensemble -> R2: {r2_mean:.4f}, RMSE: {rmse_mean:.4f}")

Generating OOF preds (this may take time)...
Simple average ensemble -> R2: 0.7692, RMSE: 0.6538


In [52]:
# 3) Stacking: train a Ridge meta-learner on the OOF stack
meta = Ridge(alpha=1.0)
meta.fit(stack_oof, y)                 # training on OOF preds is OK (no leakage)
ens_stack = meta.predict(stack_oof)    # predictions on the same OOF matrix
r2_stack = r2_score(y, ens_stack)
rmse_stack = np.sqrt(mean_squared_error(y, ens_stack))
print(f"Stacking (Ridge) ensemble -> R2: {r2_stack:.4f}, RMSE: {rmse_stack:.4f}")

# Show meta-learner weights
print("Meta-learner coefficients (weights):", meta.coef_)
print("Meta-learner intercept:", meta.intercept_)

Stacking (Ridge) ensemble -> R2: 0.7702, RMSE: 0.6523
Meta-learner coefficients (weights): [0.30756501 0.27090575 0.4563272 ]
Meta-learner intercept: -0.20602609396310445


In [53]:
joblib.dump(meta, "ensemble_meta_ridge.joblib")
print("Saved final base pipelines and meta-learner.")

Saved final base pipelines and meta-learner.
